In [10]:
import pandas as pd
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import audioFeatureExtraction
from pydub import AudioSegment
import sys,json, os 
import numpy as np
import pydub
import librosa

def stats(matrix):
    mean=np.mean(matrix)
    std=np.std(matrix)
    maxv=np.amax(matrix)
    minv=np.amin(matrix)
    median=np.median(matrix)

    output=np.array([mean,std,maxv,minv,median])
    
    return output

def convert_mono(filename):
    mono=filename[0:-4]+'_mono.wav'
    os.system('ffmpeg -i %s -ac 1 %s'% (filename,mono))
    return mono

def pyaudio_featurize(file): 
    filename=file;
    mono=convert_mono(filename)
    [Fs, x] = audioBasicIO.readAudioFile(mono)
    F = audioFeatureExtraction.stFeatureExtraction(x, Fs, 0.050*Fs, 0.025*Fs);
    os.remove(mono)

    data={
        'features': F[0].tolist()
        }

    jsonfile=open(filename[0:-4]+'.json','w')
    print(jsonfile)
    json.dump(data,jsonfile)
    jsonfile.close() 
    
    jsonfile=file[0:-4]+'.json'
    g=json.load(open(jsonfile))
    features=np.array(g['features'])
    
    new_features=list()
    all_labels=['zero crossing rate','energy','entropy of energy','spectral centroid',
                'spectral spread', 'spectral entropy', 'spectral flux', 'spectral rolloff',
                'mfcc1','mfcc2','mfcc3','mfcc4',
                'mfcc5','mfcc6','mfcc7','mfcc8',
                'mfcc9','mfcc10','mfcc11','mfcc12',
                'mfcc13','chroma1','chroma2','chroma3',
                'chroma4','chroma5','chroma6','chroma7',
                'chroma8','chroma9','chroma10','chroma11',
                'chroma12','chroma deviation']
    labels=list()
                
    for i in range(len(features)):
        tfeature=stats(features[i])
        for j in range(len(tfeature)):
            new_features.append(tfeature[j])
            if j==0:
                labels.append('mean '+all_labels[i])
            elif j==1:
                labels.append('std '+all_labels[i])
            elif j==2:
                labels.append('max '+all_labels[i])
            elif j==3:
                labels.append('min '+all_labels[i])
            elif j==4:
                labels.append('median '+all_labels[i])
            
    new_features=np.array(new_features)
    os.remove(jsonfile)
    
    return new_features, labels

In [12]:
predictor_df = pd.read_csv('Desktop\\data-sample.csv')
predictor_df.head()

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-train/sample-000000.mp3,learn to recognize omens and follow them the o...,1,0,NaN,NaN,NaN,NaN
1,cv-valid-train/sample-000001.mp3,everything in the universe evolved he said,1,0,NaN,NaN,NaN,NaN
2,cv-valid-train/sample-000002.mp3,you came so that you could learn about your dr...,1,0,NaN,NaN,NaN,NaN
3,cv-valid-train/sample-000003.mp3,so now i fear nothing because it was those ome...,1,0,NaN,NaN,NaN,NaN
4,cv-valid-train/sample-000004.mp3,if you start your emails with greetings let me...,3,2,NaN,NaN,NaN,NaN


In [13]:
predictor_df['filename'] = predictor_df['filename'].apply(lambda s: s[15:])

In [14]:
predictor_df = predictor_df[['filename','age']]
predictor_df.dropna(inplace=True)
predictor_df.head()

,filename,age
5,sample-000005.mp3,twenties
8,sample-000008.mp3,seventies
13,sample-000013.mp3,thirties
14,sample-000014.mp3,sixties
19,sample-000019.mp3,fifties


In [15]:
lst = []
extract_labels = pyaudio_featurize('D:\\data-sample\\sample-000005.mp3')
labels = extract_labels[1]
for sample in list(predictor_df['filename']):
    features = pyaudio_featurize('D:\\data-sample\\%s'%(sample))
    row = list(features[0])
    row.insert(0,sample)
    lst.append(row)
    
labels.insert(0,'filename')    
df = pd.DataFrame(lst,columns=labels)

<_io.TextIOWrapper name='D:\\data-sample\\sample-000005.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='D:\\data-sample\\sample-000005.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='D:\\data-sample\\sample-000008.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='D:\\data-sample\\sample-000013.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='D:\\data-sample\\sample-000014.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='D:\\data-sample\\sample-000019.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='D:\\data-sample\\sample-000020.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='D:\\data-sample\\sample-000023.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='D:\\data-sample\\sample-000026.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='D:\\data-sample\\sample-000027.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='D:\\data-sample\\sample-000029.json' mode='w' encoding='cp1252'>
<_io.TextI

In [18]:
df.head()

,filename,mean zero crossing rate,std zero crossing rate,max zero crossing rate,min zero crossing rate,median zero crossing rate,mean energy,std energy,max energy,min energy,...,mean chroma12,std chroma12,max chroma12,min chroma12,median chroma12,mean chroma deviation,std chroma deviation,max chroma deviation,min chroma deviation,median chroma deviation
0,sample-000005.mp3,0.047789,0.042478,0.186328,0.006669,0.036265,0.022924,0.037747,0.220714,1.523877e-06,...,0.007217,0.008824,0.047891,4.116759e-05,0.003949,0.029210,0.017521,0.086393,0.001031,0.025665
1,sample-000008.mp3,0.088419,0.072179,0.249687,0.017924,0.048145,0.011318,0.022113,0.118488,4.598946e-07,...,0.010880,0.016489,0.110760,1.066240e-04,0.005028,0.013903,0.009120,0.038503,0.000384,0.012327
2,sample-000013.mp3,0.055797,0.034378,0.203835,0.010004,0.048770,0.007959,0.014589,0.081280,1.481831e-07,...,0.012655,0.024785,0.138197,7.509763e-05,0.003206,0.019768,0.014402,0.060024,0.001206,0.015089
3,sample-000014.mp3,0.045547,0.048840,0.234264,0.005836,0.019591,0.019800,0.035730,0.243283,1.317588e-07,...,0.004765,0.011139,0.123767,5.046712e-05,0.001358,0.034290,0.016110,0.075657,0.000615,0.031811
4,sample-000019.mp3,0.047690,0.054101,0.309712,0.000000,0.023760,0.011977,0.016252,0.074011,2.609516e-10,...,0.003470,0.006089,0.040693,1.589623e-35,0.001410,0.040195,0.018742,0.088944,0.001674,0.038025


We merge the two dataframes on the column 'filename' so we can create a DataFrame with the audio features, as well as the age category.

In [19]:
new_df = df.merge(predictor_df,on='filename')

In [20]:
new_df

,filename,mean zero crossing rate,std zero crossing rate,max zero crossing rate,min zero crossing rate,median zero crossing rate,mean energy,std energy,max energy,min energy,...,std chroma12,max chroma12,min chroma12,median chroma12,mean chroma deviation,std chroma deviation,max chroma deviation,min chroma deviation,median chroma deviation,age
0,sample-000005.mp3,0.047789,0.042478,0.186328,0.006669,0.036265,0.022924,0.037747,0.220714,1.523877e-06,...,0.008824,0.047891,4.116759e-05,0.003949,0.029210,0.017521,0.086393,0.001031,0.025665,twenties
1,sample-000008.mp3,0.088419,0.072179,0.249687,0.017924,0.048145,0.011318,0.022113,0.118488,4.598946e-07,...,0.016489,0.110760,1.066240e-04,0.005028,0.013903,0.009120,0.038503,0.000384,0.012327,seventies
2,sample-000013.mp3,0.055797,0.034378,0.203835,0.010004,0.048770,0.007959,0.014589,0.081280,1.481831e-07,...,0.024785,0.138197,7.509763e-05,0.003206,0.019768,0.014402,0.060024,0.001206,0.015089,thirties
3,sample-000014.mp3,0.045547,0.048840,0.234264,0.005836,0.019591,0.019800,0.035730,0.243283,1.317588e-07,...,0.011139,0.123767,5.046712e-05,0.001358,0.034290,0.016110,0.075657,0.000615,0.031811,sixties
4,sample-000019.mp3,0.047690,0.054101,0.309712,0.000000,0.023760,0.011977,0.016252,0.074011,2.609516e-10,...,0.006089,0.040693,1.589623e-35,0.001410,0.040195,0.018742,0.088944,0.001674,0.038025,fifties
5,sample-000020.mp3,0.130399,0.098695,0.387245,0.013756,0.091913,0.015007,0.030234,0.179648,2.817492e-08,...,0.015408,0.107128,3.305850e-05,0.001905,0.013802,0.013480,0.065002,0.000455,0.011424,thirties
6,sample-000023.mp3,0.061479,0.041055,0.252188,0.007503,0.055023,0.010037,0.018699,0.086927,1.554676e-08,...,0.016390,0.126416,1.676309e-04,0.007142,0.016292,0.010535,0.051789,0.000463,0.014435,thirties
7,sample-000026.mp3,0.081581,0.068590,0.323885,0.007086,0.057524,0.018969,0.024416,0.140501,7.195277e-06,...,0.005361,0.055256,8.465308e-05,0.001866,0.023713,0.015831,0.071556,0.000566,0.020954,twenties
8,sample-000027.mp3,0.074063,0.067816,0.314714,0.009170,0.050438,0.007949,0.013128,0.065992,1.102700e-08,...,0.011129,0.061873,2.218868e-05,0.004306,0.013513,0.008746,0.041239,0.000688,0.012692,twenties
9,sample-000029.mp3,0.043102,0.058822,0.280534,0.000000,0.022093,0.016526,0.018824,0.092683,1.176604e-10,...,0.010234,0.075700,8.606310e-36,0.002428,0.024928,0.013653,0.070007,0.001222,0.022736,thirties


In [22]:
new_df.to_csv('Desktop\\audio_training_data_cleaned.csv') 